In [36]:
import pandas as pd
import numpy as np
from __future__ import print_function

**gff 파일 입력**

In [2]:
basedir = '/data2/k821209/pascal/droughtnet/'
file_gff_ath = basedir + 'Athaliana_167_gene.gff3'
file_gff_esa = basedir + 'Thalophila_173_gene.gff3'

In [3]:
df_ath = pd.read_csv(file_gff_ath,sep='\t',header=None,skiprows=1)
df_esa = pd.read_csv(file_gff_esa,sep='\t',header=None,skiprows=1)

In [4]:
mask = (df_ath[2]=='mRNA')
df_ath = df_ath[mask]
#df_ath.head()

In [5]:
mask = (df_esa[2]=='mRNA')
df_esa = df_esa[mask]
#df_tha.head()

In [6]:
def get_genename(query):
    dic = dict(zip([x.split('=')[0] for x in query.split(';')],[x.split('=')[1] for x in query.split(';')]))
    return(dic['Name'])

In [7]:
ath_genename_list = [get_genename(x) for x in df_ath[8]]
df_ath["genename"] = ath_genename_list
df_ath = df_ath.set_index('genename')
esa_genename_list = [get_genename(x) for x in df_esa[8]]
df_esa["genename"] = esa_genename_list
df_esa = df_esa.set_index('genename')

In [216]:
df_ath.head()
df_esa.head()

,0,1,2,3,4,5,6,7,8
genename,,,,,,,,,
Thhalv10024176m,scaffold_1,phytozome9_0,mRNA,10215584,10239664,.,+,.,ID=PAC:20194900;Name=Thhalv10024176m;pacid=201...
Thhalv10024178m,scaffold_1,phytozome9_0,mRNA,10222202,10239399,.,+,.,ID=PAC:20194901;Name=Thhalv10024178m;pacid=201...
Thhalv10024177m,scaffold_1,phytozome9_0,mRNA,1677535,1695178,.,+,.,ID=PAC:20192943;Name=Thhalv10024177m;pacid=201...
Thhalv10024179m,scaffold_1,phytozome9_0,mRNA,488357,503965,.,+,.,ID=PAC:20194524;Name=Thhalv10024179m;pacid=201...
Thhalv10024180m,scaffold_1,phytozome9_0,mRNA,3576033,3586074,.,-,.,ID=PAC:20194081;Name=Thhalv10024180m;pacid=201...


**mcscan output 입력**

In [213]:
file_kaks = '/data2/k821209/pascal/droughtnet/At2Th.collinearity.kaks.clean'
df_kaks = pd.read_csv(file_kaks,sep='\t',header=None)

In [214]:
df_kaks['synteny_ID'] = df_kaks[0].apply(lambda x:x.strip().split('-')[0]) 

**첫 synteny ID**

In [287]:
synteny_ID = '106'

mask = (df_kaks['synteny_ID'] == synteny_ID)
ath_genelist_synteny, esa_genelist_synteny = df_kaks[mask][1].values, df_kaks[mask][2].values

ath_chr = df_ath.loc[ath_genelist_synteny[0]][0]
esa_chr = df_esa.loc[esa_genelist_synteny[0]][0]
ath_pos = [df_ath.loc[x][3] for x in ath_genelist_synteny]
esa_pos = [df_esa.loc[x][3] for x in esa_genelist_synteny]
esa_pos.sort()
mask = (df_ath[0] == ath_chr) & (int(ath_pos[0]) <= df_ath[3].astype(np.int))&( df_ath[3].astype(np.int)<= int(ath_pos[-1]))#<= ath_pos[-1]) 
ath_genelist_synteny_all = df_ath[mask].index
print(len(ath_genelist_synteny_all))

mask = (df_esa[0] == esa_chr) & (int(esa_pos[0]) <= df_esa[3].astype(np.int))&( df_esa[3].astype(np.int)<= int(esa_pos[-1]))#<= ath_pos[-1]) 
esa_genelist_synteny_all = df_esa[mask].index
print(len(esa_genelist_synteny_all))
#len(esa_genelist_synteny_all)
#print( esa_genelist_synteny_all[0:10])
#esa_genelist_synteny_all = esa_genelist_synteny_all[::-1]
#print(esa_genelist_synteny_all[0:10])

diameter = 5 # of the circle representing a gene
spacing = 3 # fixed distance between genes 
ath_line_length = (diameter*2 + spacing) * len(ath_genelist_synteny_all)
esa_line_length = (diameter*2 + spacing) * len(esa_genelist_synteny_all)

Outfile = open('synteny_ID_%s.svg'%synteny_ID,'w')
width = ath_line_length + 200
height = 1000
gap = 300 # gap between species lines
print('''<html>
<body>

<h1>My first SVG</h1>
<svg height="%d" width="%d">'''%(height,width),file=Outfile)

#line = '<line x1="%d" y1="%d" x2="%d" y2="%d" style="stroke:rgb(255,0,0);stroke-width:1" />' 
line = '<line x1="%d" y1="%d" x2="%d" y2="%d" stroke="%s" stroke-width="1" />' 
# left dot to right dot
circle = '<circle cx="%d" cy="%d" r="%d" stroke="black" stroke-width="1" fill="%s" fill-opacity="0.4"/>'
# x, y, diameter
text =  '<text x="%d" y="%d" fill="black">%s</text>'
# x, y, text

# Line! 
Alx = (width/2)-(ath_line_length/2)
Aly = 20
Arx = (width/2)+(ath_line_length/2)
Ary = 20
print(line%(Alx,Aly,Arx,Ary,'grey'),file=Outfile)
print(text%(Alx,Aly-10,ath_chr),file=Outfile)

# Line! 
Blx = (width/2)-(esa_line_length/2)
Bly = Aly + gap
Brx = (width/2)+(esa_line_length/2)
Bry = Ary + gap
print(line%(Blx,Bly,Brx,Bry,'grey'),file=Outfile)
print(text%(Blx,Bly-10,esa_chr),file=Outfile)

dicG2P_ath = {} # genename to position
for i,j  in enumerate(ath_genelist_synteny_all):
    print(circle%(i*(spacing+diameter*2)+Alx,Aly,diameter,'white'),file=Outfile)
    dicG2P_ath[j] = [i*(spacing+diameter*2)+Alx,Aly]

dicG2P_esa = {}
for i,j  in enumerate(esa_genelist_synteny_all):
    print(circle%(i*(spacing+diameter*2)+Blx,Bly,diameter,'white'),file=Outfile)
    dicG2P_esa[j] = [i*(spacing+diameter*2)+Blx,Bly]

for i,j in enumerate(ath_genelist_synteny):
    gnA = j
    gnB = esa_genelist_synteny[i]
    lx, ly = dicG2P_ath[gnA]
    rx, ry = dicG2P_esa[gnB]
    print(line%(lx,ly,rx,ry,'grey'),file=Outfile)
    print(circle%(lx,ly,diameter,'red'),file=Outfile)
    print(circle%(rx,ry,diameter,'red'),file=Outfile)

1669
1429


**두번째 부터는 다음 펑션으로**

In [288]:
def add_synteny(ID,layer,start):
    synteny_ID = ID

    mask = (df_kaks['synteny_ID'] == synteny_ID)
    ath_genelist_synteny, esa_genelist_synteny = df_kaks[mask][1].values, df_kaks[mask][2].values

    ath_chr = df_ath.loc[ath_genelist_synteny[0]][0]
    esa_chr = df_esa.loc[esa_genelist_synteny[0]][0]
    ath_pos = [df_ath.loc[x][3] for x in ath_genelist_synteny]
    esa_pos = [df_esa.loc[x][3] for x in esa_genelist_synteny]
    esa_pos.sort()
    mask = (df_ath[0] == ath_chr) & (int(ath_pos[0]) <= df_ath[3].astype(np.int))&( df_ath[3].astype(np.int)<= int(ath_pos[-1]))#<= ath_pos[-1]) 
    ath_genelist_synteny_all = df_ath[mask].index
    print(len(ath_genelist_synteny_all))

    mask = (df_esa[0] == esa_chr) & (int(esa_pos[0]) <= df_esa[3].astype(np.int))&( df_esa[3].astype(np.int)<= int(esa_pos[-1]))#<= ath_pos[-1]) 
    esa_genelist_synteny_all = df_esa[mask].index
    print(len(esa_genelist_synteny_all))
    #len(esa_genelist_synteny_all)
    #print( esa_genelist_synteny_all[0:10])
    #esa_genelist_synteny_all = esa_genelist_synteny_all[::-1]
    #print(esa_genelist_synteny_all[0:10])

    #diameter = 5 # of the circle representing a gene
    #spacing = 3 # fixed distance between genes 
    ath_line_length = (diameter*2 + spacing) * len(ath_genelist_synteny_all)
    esa_line_length = (diameter*2 + spacing) * len(esa_genelist_synteny_all)


    #height = 500
    gap = 300 # gap between species lines
    # Line! 
    Blx = start
    Bly = Aly + gap * layer
    Brx = start+esa_line_length
    Bry = Ary + gap * layer
    print(line%(Blx,Bly,Brx,Bry,'grey'),file=Outfile)
    print(text%(Blx,Bly-10,esa_chr+':'+str(esa_pos[0])+'..'+str(esa_pos[-1])),file=Outfile)

    dicG2P_ath = {} # genename to position
    for i,j  in enumerate(ath_genelist_synteny_all):
        print(circle%(i*(spacing+diameter*2)+Alx,Aly,diameter,'white'),file=Outfile)
        dicG2P_ath[j] = [i*(spacing+diameter*2)+Alx,Aly]
    
    dicG2P_esa = {}
    for i,j  in enumerate(esa_genelist_synteny_all):
        print(circle%(i*(spacing+diameter*2)+Blx,Bly,diameter,'white'),file=Outfile)
        dicG2P_esa[j] = [i*(spacing+diameter*2)+Blx,Bly]

    for i,j in enumerate(ath_genelist_synteny):
        gnA = j
        gnB = esa_genelist_synteny[i]
        lx, ly = dicG2P_ath[gnA]
        rx, ry = dicG2P_esa[gnB]
        print(line%(lx,ly,rx,ry,'grey'),file=Outfile)
        print(circle%(lx,ly,diameter,'red'),file=Outfile)
        print(circle%(rx,ry,diameter,'red'),file=Outfile)

In [289]:
add_synteny('51',2,width/4)
add_synteny('72',2,width/4-500)
add_synteny('78',3,width/5)
add_synteny('79',3,width/5+1000)


557
358
30
33
70
63
70
73


In [290]:
print('''</svg>

</body>
</html>
''',file=Outfile)

Outfile.close()